

---

# Part 3: The Network and Transport Layers – Addressing, Routing, and Reliable Delivery

## Chapter 6: The Network Layer (L3): IP Addressing (IPv4)

In the preceding chapters, we built a solid foundation. We explored the physical transmission of bits (Layer 1) and how those bits are organized into frames for local delivery using MAC addresses and switches (Layer 2). We now have the ability to communicate with any device on our own local network segment. But the world is much larger than a single LAN.

This is where the **Network Layer**, Layer 3 of the OSI model, comes into play. Its purpose is profound: to enable communication between devices that are **not** on the same physical network. It provides the mechanisms for **end-to-end delivery** across an internetwork—a network of networks. This is the layer that makes the Internet possible.

The dominant protocol at this layer is the **Internet Protocol (IP)** . This chapter will focus on the version that has powered the Internet for decades: **IPv4 (Internet Protocol version 4)** . You will learn about the structure of IPv4 addresses, the crucial role of the subnet mask, the function of the default gateway, and the concepts that underpin all modern network design. By the end of this chapter, you will be able to look at any IPv4 address and understand its fundamental components.

### 6.1 The Purpose of the Network Layer: End-to-End Delivery

The Network Layer has a single, overarching mission: to get a packet from a source host to a destination host, even if that destination is on the other side of the planet. To accomplish this, it performs several key functions:

1.  **Logical Addressing:** It defines a scheme for assigning a unique logical address to every device on an internetwork. This address, the **IP address**, is hierarchical. Unlike a flat MAC address, an IP address contains information about which network the device belongs to. This hierarchical structure is what allows routers to efficiently forward packets without needing to know the location of every single host.

2.  **Routing:** This is the core intelligence of the Network Layer. It is the process of finding the best path through a series of interconnected networks (routers) for a packet to reach its destination. Routers, the primary devices at this layer, maintain **routing tables** that contain information about networks, not individual hosts.

3.  **Packetizing:** It encapsulates the data it receives from the Transport Layer (Layer 4) into **packets** (also called datagrams). It adds a header that contains, among other things, the source and destination IP addresses.

4.  **Fragmentation and Reassembly:** If a packet is too large for the maximum transmission unit (MTU) of a particular link along the path, the Network Layer can fragment the packet into smaller pieces. These fragments are then reassembled at the destination.

It is vital to understand the distinction between the Data Link Layer and the Network Layer:
- **Data Link Layer (L2):** Delivers data from **node to node** on the same network. It uses MAC addresses and is unaware of the larger internetwork.
- **Network Layer (L3):** Delivers data from the **source host to the destination host** across an internetwork. It uses IP addresses and relies on routers to forward packets between networks.

### 6.2 The IPv4 Address Structure: 32-bit Dotted Decimal Notation

An IPv4 address is a 32-bit binary number. That means it is a string of 32 ones and zeros, such as `11000000101010000000000100000001`. While this is how the computer sees it, it is not very user-friendly. To make it easier for humans, we use **dotted decimal notation**.

In dotted decimal, the 32-bit address is divided into four 8-bit sections, called **octets**. Each octet is converted from binary to its decimal equivalent, and the four decimal numbers are separated by dots.

For example, the binary address `11000000101010000000000100000001` is broken down as follows:

- Octet 1: `11000000` = 192
- Octet 2: `10101000` = 168
- Octet 3: `00000001` = 1
- Octet 4: `00000001` = 1

So, the dotted decimal representation is **`192.168.1.1`** .

This is the format you are familiar with. Because each octet is 8 bits, the value in each octet can range from 0 to 255. Therefore, a valid IPv4 address can range from `0.0.0.0` to `255.255.255.255`.

### 6.3 The Subnet Mask: Separating Network and Host

If an IP address is just a unique identifier, how does a device or a router know which part of the address identifies the **network** and which part identifies the specific **host** on that network? This is the job of the **subnet mask**.

The subnet mask is a 32-bit number, also written in dotted decimal, that is paired with every IP address. Its function is to "mask" the IP address, separating the network portion from the host portion. It works through a logical AND operation.

The subnet mask consists of a sequence of consecutive 1s followed by a sequence of consecutive 0s. The 1s indicate the bits of the IP address that represent the **network portion**. The 0s indicate the bits that represent the **host portion**.

Consider the most common subnet mask for home and small office networks: `255.255.255.0`. In binary, this is:

```
11111111 . 11111111 . 11111111 . 00000000
```

Now, let's apply this to our example IP address `192.168.1.1`:

- **IP Address (Binary):** `11000000 . 10101000 . 00000001 . 00000001`
- **Subnet Mask (Binary):** `11111111 . 11111111 . 11111111 . 00000000`
- **Logical AND Result:** `11000000 . 10101000 . 00000001 . 00000000`

The result of the AND operation, converted back to decimal, is **`192.168.1.0`** . This is the **network address**.

This tells us that, with a subnet mask of `255.255.255.0`, the first three octets (`192.168.1`) identify the network, and the last octet (the one that became all zeros in the AND operation) identifies the host. Therefore, all devices with an IP address starting with `192.168.1.` are on the same local network.

**Key Takeaway:** You cannot determine the network and host portions of an IP address by looking at the address alone. You *must* have the subnet mask. The combination of the IP address and the subnet mask defines the network.

### 6.4 The Default Gateway: The Exit from the LAN

We have a device with an IP address and a subnet mask. It knows its own network address. Now, it needs to communicate with another device. What happens if that device is not on the same local network?

This is where the **default gateway** comes in. The default gateway is the IP address of the router that connects the local network to other networks (such as the Internet). It is the "door" or "exit" from the LAN.

When your computer wants to send data to an IP address, it performs the following logic:

1.  It performs the logical AND operation using its own IP address and subnet mask to determine its own network address (e.g., `192.168.1.0`).
2.  It performs the same AND operation using the **destination IP address** and its own subnet mask.
3.  It compares the two results.
    - If the network addresses **match**, the destination is on the **same local network**. Your computer will use ARP to find the destination's MAC address and send the frame directly.
    - If the network addresses **do not match**, the destination is on a **different network**. Your computer cannot deliver the frame directly. Instead, it will send the frame to the default gateway. The destination MAC address in the frame will be the MAC address of the default gateway (obtained via ARP), not the ultimate destination. The gateway router then takes over, stripping off the frame, reading the IP packet, and forwarding it towards its final destination.

The default gateway is a critical piece of configuration. Without a correctly configured default gateway, your device can only communicate with other devices on its own local network.

### 6.5 Public vs. Private IP Addresses (RFC 1918)

As the Internet grew, it became clear that the 4.3 billion addresses provided by IPv4 would not be enough for every device in the world to have its own globally unique, publicly routable address. To conserve address space, the Internet Engineering Task Force (IETF) defined blocks of IP addresses that would be reserved for private use, never to be routed on the public Internet. These are defined in **RFC 1918**.

The three blocks of private IP addresses are:

- **Class A:** `10.0.0.0` to `10.255.255.255` (with a subnet mask of `255.0.0.0` or /8). This provides a single large block of over 16 million addresses, typically used by large organizations.
- **Class B:** `172.16.0.0` to `172.31.255.255` (with a subnet mask of `255.240.0.0` or /12). This provides 16 contiguous blocks, often used by medium-sized organizations.
- **Class C:** `192.168.0.0` to `192.168.255.255` (with a subnet mask of `255.255.0.0` or /16). This provides 256 blocks of 256 addresses, commonly used in homes and small businesses.

**Characteristics of Private IP Addresses:**

- **Non-Routable on the Internet:** Routers on the public Internet are configured to discard any packets destined for a private IP address. This is a key security feature.
- **Reusable:** Any organization can use these addresses internally. Your home network uses `192.168.1.x`, and so does your neighbor's. They will never conflict because they are isolated on their own private networks.
- **Must Be Translated:** For a device with a private IP address to communicate with the public Internet, its address must be translated to a public IP address. This is done by **Network Address Translation (NAT)** , typically performed by your home router.

In contrast, a **public IP address** is an address that is globally unique and can be routed on the Internet. These addresses must be registered with a Regional Internet Registry (RIR) and are assigned to organizations and Internet Service Providers (ISPs). When you access a website like `google.com`, your packets are sent to the public IP address of Google's servers.

### 6.6 Introduction to Network Address Translation (NAT): Conserving Public IPs

**Network Address Translation (NAT)** is the process of modifying IP address information in packet headers while in transit. Its primary purpose, especially in the context of IPv4 conservation, is to allow multiple devices on a private network to share a single public IP address to access the Internet.

The most common form of NAT is called **PAT (Port Address Translation)** , often referred to as "NAT Overload." This is what your home router does.

**How NAT (PAT) Works:**

1.  Your laptop, with a private IP of `192.168.1.100`, wants to access a web server at public IP `203.0.113.5` on port 80. It creates a packet with:
    - Source IP: `192.168.1.100`, Source Port: `12345` (a random ephemeral port)
    - Destination IP: `203.0.113.5`, Destination Port: `80`

2.  The packet arrives at your home router. The router's NAT table is empty. It sees this is a new connection.

3.  The router changes the packet before sending it out to the Internet. It:
    - Replaces the private source IP address (`192.168.1.100`) with its own public IP address (e.g., `198.51.100.20`).
    - Replaces the source port (`12345`) with a different, unique port number from its own pool (e.g., `50001`).

4.  The router creates an entry in its **NAT table** that maps the internal, private connection to the external, public one. The entry looks something like this:
    - `Internal: 192.168.1.100:12345 <-> External: 198.51.100.20:50001`

5.  The modified packet is sent to the web server. To the server, it looks like the request came directly from the router at `198.51.100.20:50001`.

6.  The web server sends its response back to the router's public IP address, addressed to port `50001`.

7.  The router receives the response. It looks at the destination port (`50001`) in its NAT table and finds the corresponding internal device and port (`192.168.1.100:12345`).

8.  The router reverses the translation, changing the destination IP and port back to the laptop's private address and original port, and forwards the packet to the laptop.

To the laptop and the web server, it appears as if they are communicating directly. The entire translation process is transparent. This single mechanism has been the primary driver of IPv4's longevity, allowing millions of devices to share a much smaller pool of public IP addresses.

### 6.7 Special Addresses: Loopback, Broadcast, Network Address

Within the IPv4 addressing scheme, certain addresses have special meanings and are reserved for specific functions.

- **Network Address:** As we saw with the subnet mask, the address where all the host bits are set to `0` is the **network address**. This address is used to identify the network itself and cannot be assigned to a device's interface. For the network `192.168.1.0/24`, the network address is `192.168.1.0`.

- **Broadcast Address:** The address where all the host bits are set to `1` is the **broadcast address**. A packet sent to this address is intended to be received by **all devices** on that local network. For the network `192.168.1.0/24`, the broadcast address is `192.168.1.255`. When your computer sends an ARP request (as we saw in Chapter 5), it is sent to the Layer 2 broadcast address, but the ARP packet itself is destined for the Layer 3 broadcast address of the local network.

- **Loopback Address:** The entire `127.0.0.0/8` network is reserved for loopback, but the most commonly used address is **`127.0.0.1`** . This is a special address that a device uses to refer to itself. Packets sent to `127.0.0.1` are treated as if they are network packets, but they are processed internally by the TCP/IP stack and never sent out onto the physical network. It is used for testing network software and services locally. When you `ping 127.0.0.1`, you are testing to see if your computer's TCP/IP stack is working correctly. This address is often referred to as "localhost."

- **Default Route:** The address `0.0.0.0` is used in routing tables to represent the **default route**. It essentially means "any network not explicitly listed in the routing table." When a router doesn't have a specific route to a destination, it will forward the packet to the next hop specified by the default route. On your computer, the default route is configured via the default gateway.

---

### Chapter 6: Hands-On Challenge

Let's explore these concepts on your own machine.

1.  **Examine Your IP Configuration:**
    - Open your command line or terminal.
    - Type `ipconfig /all` (Windows) or `ifconfig` (macOS/Linux).
    - Identify your **IPv4 Address**, your **Subnet Mask**, and your **Default Gateway**.
    - Write them down. For example:
        - IPv4 Address: `192.168.1.105`
        - Subnet Mask: `255.255.255.0`
        - Default Gateway: `192.168.1.1`

2.  **Calculate Your Network and Broadcast Addresses:**
    - Based on your IP and subnet mask, what is your **network address**? (Hint: Set all host bits to 0).
    - What is your **broadcast address**? (Hint: Set all host bits to 1).
    - *Check your work:* In the `192.168.1.105/24` example, the network address is `192.168.1.0` and the broadcast is `192.168.1.255`.

3.  **Test the Loopback:**
    - In your command line, type `ping 127.0.0.1` and press Enter.
    - You should see replies from your own computer. This confirms that your internal networking software is functioning correctly.

4.  **Trace a Packet to an External Destination:**
    - We will use a tool called `tracert` (Windows) or `traceroute` (macOS/Linux) to see the path a packet takes to a remote server. Type:
        - Windows: `tracert 8.8.8.8`
        - macOS/Linux: `traceroute 8.8.8.8`
    - `8.8.8.8` is a public DNS server operated by Google.
    - The first line of output should show your **default gateway**. This is the first hop—your router. The subsequent lines show the routers (Layer 3 devices) that your packet traverses as it leaves your network and crosses the Internet to reach its destination. You are watching the Network Layer in action.

5.  **Determine if Your IP is Public or Private:**
    - Look at your IP address from step 1. Is it in one of the RFC 1918 private ranges (`10.x.x.x`, `172.16.x.x` to `172.31.x.x`, or `192.168.x.x`)? It almost certainly is.
    - Now, find your public IP address. A quick way is to search "what is my ip" on a web search engine. You will see a different address. This is the public IP address assigned to your router by your ISP and shared by all your devices through NAT.

---

You have now taken a major step. You understand the fundamental addressing scheme of the Internet. You know the difference between a local network and a remote one, and how your device uses the default gateway to reach the outside world.

In the next chapter, we will build on this knowledge with one of the most essential and practical skills in all of networking: **The Art of Subnetting and CIDR**. You will learn how to take a block of IP addresses and divide it efficiently to meet the needs of any network, large or small.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../2. the_physical_and_data_link_layers_the_hardware_and_local_delivery/5. the_data_link_layer_the_nic_switching_and_the_mac_address.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='7. the_art_of_subnetting_and_cidr.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
